In [273]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import plotly.express as px

In [274]:
discharge_path = '/Users/kkkg0829/Desktop/project/data set/d_B0005_lf.csv'
discharge_df = pd.read_csv(discharge_path)

In [275]:
# 1. feature matrix
X = discharge_df[features]

In [276]:
# 2. scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [277]:
# 3. Isolation Forest 학습
model = IsolationForest(
    n_estimators=300,
    max_features=1.0,
    contamination=0.01,   # 이상치 비율 
    n_jobs=-1,
    random_state=42
)
model.fit(X_scaled)

,n_estimators,300
,max_samples,'auto'
,contamination,0.01
,max_features,1.0
,bootstrap,False
,n_jobs,-1
,random_state,42
,verbose,0
,warm_start,False


In [278]:
# 4. predict (1=normal, -1=anomaly)
y_pred = model.predict(X_scaled)

In [279]:
# 5. anomaly score (값 클수록 더 이상)
scores = model.score_samples(X_scaled)
anomaly_score = -scores

In [280]:
# 6. contamination이 아닌 score 기반 threshold
threshold = np.quantile(anomaly_score, 0.99)   # 상위 1%
anomaly_label = (anomaly_score >= threshold).astype(int)

In [281]:
# 7. 결과 저장
discharge_df['anomaly_label_if'] = y_pred          # IsolationForest label (1/-1)
discharge_df['anomaly_score'] = anomaly_score      # 연속적 score
discharge_df['anomaly_label_qt'] = anomaly_label   # quantile label (0/1)

In [282]:
print(f"IsolationForest 기반 이상치 개수: {np.sum(y_pred==-1)}")
print(f"Quantile 기반 이상치 개수:       {np.sum(anomaly_label==1)}")

IsolationForest 기반 이상치 개수: 503
Quantile 기반 이상치 개수:       503


### ========================= 파생피쳐 사용 O ============================

In [283]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.ensemble import IsolationForest
from sklearn.tree import DecisionTreeClassifier

In [284]:
discharge_path = '/Users/kkkg0829/Desktop/project/data set/B0005_with_lowess_features.csv'
df = pd.read_csv(discharge_path)

In [285]:
# 모델 학습 (cycle_idx 제외)
features = [
    'Voltage_measured', 'Current_measured', 'Temperature_measured',
    'Current_load', 'Voltage_load', 'Voltage_measured_smooth',
    'Voltage_measured_residual', 'Voltage_measured_trend',
    'Current_measured_smooth', 'Current_measured_residual',
    'Current_measured_trend', 'Temperature_measured_smooth',
    'Temperature_measured_residual', 'Temperature_measured_trend',
    'Current_load_smooth', 'Current_load_residual', 'Current_load_trend',
    'Voltage_load_smooth', 'Voltage_load_residual', 'Voltage_load_trend'
]
X = df[features]

iso_model = IsolationForest(contamination=0.01, random_state=42)
df['anomaly'] = iso_model.fit_predict(X)

In [286]:
# 원인 분석 (어떤 피쳐가 이상치에 영향을 줬는가?)
# 이상치(-1)와 정상(1)을 구분하는 Decision Tree를 학습시켜 중요도 추출
explainer = DecisionTreeClassifier(max_depth=4, random_state=42)
explainer.fit(X, df['anomaly'])

# 중요도 DataFrame 생성
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': explainer.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [287]:
# 시각화
# 정상 데이터와 이상치 데이터 분리
anomalies = df[df['anomaly'] == -1]

fig = go.Figure()

# (1) 전체 흐름 - '전압(Voltage)' 기준
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df['Voltage_measured'],
    mode='lines',
    name='정상 데이터 흐름',
    line=dict(color='lightgrey', width=1.5),
    hoverinfo='skip'
))

# (2) 이상치 (빨간 점)
fig.add_trace(go.Scatter(
    x=anomalies.index, 
    y=anomalies['Voltage_measured'],
    mode='markers',
    name='이상치(Anomaly)',
    marker=dict(color='red', size=7, symbol='x'), 
    # 마우스 올렸을 때 보여줄 정보
    customdata=anomalies[['cycle_idx', 'Temperature_measured', 'Current_measured']],
    hovertemplate=(
        "<b>[이상치 탐지]</b><br>" +
        "시간(Index): %{x}<br>" +
        "전압: %{y:.3f}V<br>" +
        "<b>사이클: %{customdata[0]}</b><br>" +
        "온도: %{customdata[1]:.2f}<br>" +
        "전류: %{customdata[2]:.3f}<extra></extra>"
    )
))

fig.update_layout(
    title='<b>배터리 이상치 탐지 시각화</b> (상위 1% 이상치)',
    xaxis_title='시간 흐름 (Time Steps)',
    yaxis_title='전압 (Voltage)',
    template='plotly_white',
    hovermode='closest',
    height=600
)

fig.show()

In [288]:
# 결과 해석 
print("\n" + "="*50)
print("   [이상치 발생 원인 분석: 피쳐 중요도]")
print("="*50)
print("어떤 변수가 튀어서 이상치로 잡혔는지 비중(%)으로 보여준다.\n")

for index, row in importance_df.head(5).iterrows():
    print(f"{row['Feature']:30} : {row['Importance']*100:.1f}% 영향")
print("="*50)


   [이상치 발생 원인 분석: 피쳐 중요도]
어떤 변수가 튀어서 이상치로 잡혔는지 비중(%)으로 보여준다.

Current_measured_residual      : 57.0% 영향
Voltage_measured_trend         : 32.9% 영향
Voltage_load_trend             : 6.6% 영향
Voltage_measured_smooth        : 1.9% 영향
Current_load_trend             : 0.6% 영향


In [289]:
# 1. 원인 분석 모델 학습 (Surrogate Model)
explainer = DecisionTreeClassifier(max_depth=4, random_state=42)
explainer.fit(X, df['anomaly'])

,criterion,'gini'
,splitter,'best'
,max_depth,4
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [290]:
# 2. 중요도 데이터프레임 만들기
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': explainer.feature_importances_
}).sort_values(by='Importance', ascending=True) 

In [291]:
# 중요도가 0인(영향 없는) 피쳐는 제거해서 깔끔하게
importance_df = importance_df[importance_df['Importance'] > 0]

In [292]:
# 3. 시각화 (가로 막대 그래프)
fig_imp = px.bar(
    importance_df, 
    x='Importance', 
    y='Feature', 
    orientation='h',
    title='이상치 발생에 영향을 준 주요 원인 (Feature Importance)',
    labels={'Importance': '영향력 (비중)', 'Feature': '피쳐 이름'},
    text_auto='.1%'
)

fig_imp.update_layout(template='plotly_white')
fig_imp.show()

### ========================== 파생피쳐 사용 X =================================

In [293]:
# 모델 학습 (cycle_idx 제외)
x_features = [
    'Voltage_measured', 'Current_measured', 'Temperature_measured',
    'Current_load', 'Voltage_load'
]
X = df[x_features]

iso_model = IsolationForest(contamination=0.01, random_state=42)
df['anomaly'] = iso_model.fit_predict(X)

In [294]:
# 원인 분석 (어떤 피쳐가 이상치에 영향을 줬는가?)
# 이상치(-1)와 정상(1)을 구분하는 Decision Tree를 학습시켜 중요도 추출
x_explainer = DecisionTreeClassifier(max_depth=4, random_state=42)
x_explainer.fit(X, df['anomaly'])

# 중요도 DataFrame 생성
x_importance_df = pd.DataFrame({
    'Feature': x_features,
    'Importance': x_explainer.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [295]:
# 시각화
# 정상 데이터와 이상치 데이터 분리
x_anomalies = df[df['anomaly'] == -1]

x_fig = go.Figure()

# (1) 전체 흐름 - '전압(Voltage)' 기준
x_fig.add_trace(go.Scatter(
    x=df.index, 
    y=df['Voltage_measured'],
    mode='lines',
    name='정상 데이터 흐름',
    line=dict(color='lightgrey', width=1.5),
    hoverinfo='skip'
))

# (2) 이상치 (빨간 점)
x_fig.add_trace(go.Scatter(
    x=x_anomalies.index, 
    y=x_anomalies['Voltage_measured'],
    mode='markers',
    name='이상치(Anomaly)',
    marker=dict(color='red', size=7, symbol='x'),
        # 마우스 올렸을 때 보여줄 정보
    customdata=x_anomalies[['cycle_idx', 'Temperature_measured', 'Current_measured']].values,
    hovertemplate=(
        "<b>[이상치 탐지]</b><br>" +
        "시간(Index): %{x}<br>" +
        "전압: %{y:.3f}V<br>" +
        "<b>사이클: %{customdata[0]}</b><br>" +
        "온도: %{customdata[1]:.2f}<br>" +
        "전류: %{customdata[2]:.3f}<extra></extra>"
    )
))

x_fig.update_layout(
    title='<b>배터리 이상치 탐지 시각화</b> (1%, 파생피쳐X)',
    xaxis_title='시간 흐름 (Time Steps)',
    yaxis_title='전압 (Voltage)',
    template='plotly_white',
    hovermode='closest',
    height=600
)

x_fig.show()

In [296]:
# 1. 원인 분석 모델 학습 (Surrogate Model)
explainer = DecisionTreeClassifier(max_depth=4, random_state=42)
explainer.fit(X, df['anomaly'])

,criterion,'gini'
,splitter,'best'
,max_depth,4
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [297]:
# 2. 중요도 데이터프레임 만들기
x_importance_df = pd.DataFrame({
    'Feature': x_features,
    'Importance': x_explainer.feature_importances_
}).sort_values(by='Importance', ascending=True) 

# 중요도가 0인(영향 없는) 피쳐는 제거해서 깔끔하게
x_importance_df = x_importance_df[x_importance_df['Importance'] > 0]